In [16]:
import geopandas as gpd
import psycopg2
import yaml
import pandas as pd
import tensorflow as tf


In [40]:
def read_yaml(file_path):
    """
    read in config file.  All options and adjustments should be made to config file.
    input: yaml file path
    output: dictionary
    """
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

In [41]:
conn_creds = read_yaml('C:\\DSSG_CHAR\\Configs\\db_conn.yaml')

connection = psycopg2.connect(
    dbname=conn_creds['dbname'],
    user=conn_creds['user'],
    password=conn_creds['password'],
    host=conn_creds['address']
)


char_query = "SELECT * FROM char_alerts;"
df = gpd.GeoDataFrame.from_postgis(char_query, connection, geom_col='centroid')

c:\Users\kylershu\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [9]:
df['loc'] = df['x_tile'].astype(str) + ',' + df['y_tile'].astype(str)
df['loc'].value_counts()
df["validation"] = df["validation"].apply(lambda x: 1 if x else 0)

In [14]:
df = df.drop_duplicates(subset='loc', keep='first')
len(df)

3085

In [26]:
result['adm_name'].isnull().sum()

np.int64(0)

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50V2
import numpy as np

# Sampling layer for the VAE
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Encoder using ResNet50V2
def build_encoder(latent_dim):
    resnet_base = ResNet50V2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    resnet_base.trainable = False
    
    x = layers.GlobalAveragePooling2D()(resnet_base.output)
    x = layers.Dense(512, activation='relu')(x)
    
    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
    
    z = Sampling()([z_mean, z_log_var])
    
    encoder = models.Model(resnet_base.input, [z_mean, z_log_var, z], name='encoder')
    return encoder

# Decoder model
def build_decoder(latent_dim):
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(7 * 7 * 256, activation='relu')(latent_inputs)
    x = layers.Reshape((7, 7, 256))(x)
    x = layers.Conv2DTranspose(128, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Conv2DTranspose(3, 3, activation='sigmoid', strides=2, padding='same')(x)
    
    decoder = models.Model(latent_inputs, x, name='decoder')
    return decoder

# VAE model
class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return reconstructed

# Build the VAE
latent_dim = 128
encoder = build_encoder(latent_dim)
decoder = build_decoder(latent_dim)
vae = VAE(encoder, decoder)

# Compile and train the model
vae.compile(optimizer='adam', loss='mse')
# Assuming you have a dataset of images loaded as `images`
# vae.fit(images, images, epochs=10, batch_size=32)

# Extract embeddings
def get_embeddings(images):
    _, _, z = encoder.predict(images)
    return z

# Example usage after training
# embeddings = get_embeddings(test_images)


In [21]:
# Set the path to your dataset
dataset_path = r'c:\DSSG_CHAR\detection_evaluation_dssg\tiles\21'

# Load images into a dataset
image_size = (256, 256)
batch_size = 32
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    label_mode=None,  # None if you don't have labels
    image_size=image_size,
    batch_size=batch_size,
    shuffle=False
)

# Normalize pixel values to [0, 1]
dataset = dataset.map(lambda x: x / 255.0)


Found 5210 files.


In [27]:
embeddings = get_embeddings(dataset)

163/163 ━━━━━━━━━━━━━━━━━━━━ 94s 576ms/step


In [28]:
embeddings.shape

(5210, 128)

In [43]:
embeddings = pd.DataFrame(embeddings)

In [44]:
import os

# Specify the folder path
folder_path = r'c:\DSSG_CHAR\detection_evaluation_dssg\tiles\21'

# Get the list of file names (excluding directories)
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Convert the list to a Pandas Series
file_names_series = pd.Series(file_names)
embeddings['file'] = file_names_series

In [49]:
embeddings['x_tile'] = embeddings['file'].str[3:8].astype(int)
embeddings['y_tile'] = embeddings['file'].str[9:14].astype(int)
embeddings['geo_month'] = embeddings['file'].str[15:17].astype(int)
embeddings['geo_year'] = embeddings['file'].str[18:20].astype(int)
embeddings

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,file,x_tile,y_tile,geo_month,geo_year
0,-1.300541,-0.838213,2.730364,0.121689,-2.364493,2.698779,-0.738184,1.739123,-0.296206,0.611519,...,-0.664175,-1.353738,-3.031446,-0.376511,-1.541445,15_11946_16870_09_23.png,11946,16870,9,23
1,1.860632,-2.484503,-0.210400,-0.281583,-2.231279,0.022268,0.883381,-0.090037,-0.528056,-0.895833,...,-0.334883,0.583236,-0.791529,-3.037896,-1.823052,15_11946_16870_10_23.png,11946,16870,10,23
2,0.937593,-1.510875,1.265654,1.858553,0.109724,0.845561,-1.204252,2.587135,0.231295,-0.543210,...,-1.085546,-1.347915,0.156867,-1.127668,1.284229,15_11946_16871_10_23.png,11946,16871,10,23
3,0.209836,-1.791340,2.908613,0.006235,0.485513,1.079943,0.058337,1.229984,-1.693903,-1.672808,...,-3.788022,-0.261990,0.646975,-0.408238,-0.716080,15_11946_16871_11_23.png,11946,16871,11,23
4,0.087534,-1.108212,2.312580,0.383922,-0.239342,1.797709,-0.413245,0.148457,0.471931,-0.034667,...,0.443268,-1.654936,0.031601,-0.539362,-0.474015,15_11959_16866_10_23.png,11959,16866,10,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,0.747319,0.739685,0.951046,2.596876,0.396434,-2.147666,-0.596616,-0.780887,1.271759,-1.851935,...,1.247949,0.062903,-0.858566,0.945114,-0.719955,15_12574_16633_07_23.png,12574,16633,7,23
5206,0.952561,0.215762,1.381521,-0.096000,-1.473336,0.513031,-0.567465,-0.498614,0.676321,-1.653897,...,1.150491,-0.365103,-0.541624,-1.219013,-0.399657,15_12574_16633_08_23.png,12574,16633,8,23
5207,-0.396212,-0.204708,1.101958,0.234761,-4.023079,-0.191049,-0.828168,0.800991,-0.063635,0.402628,...,-2.194813,0.266479,-1.666984,-0.934614,-1.583195,15_12577_16634_03_24.png,12577,16634,3,24
5208,-0.315906,-0.980185,2.384417,-0.392108,-1.967995,-1.118831,0.561503,0.877017,0.518233,0.358746,...,-6.096200,-0.592951,-1.545499,0.490262,0.527971,15_12577_16634_10_23.png,12577,16634,10,23


In [50]:
embeddings.to_parquet('embeddings.parquet')

c:\Users\kylershu\.conda\envs\dssg_dev\Lib\site-packages\pandas\io\parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
